In [1]:
!pip install selenium chromedriver-autoinstaller
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()


'c:\\Users\\ali-d\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\chromedriver_autoinstaller\\143\\chromedriver.exe'

In [ ]:
# Chrome should already be installed on Windows
# If not, please install Google Chrome manually from: https://www.google.com/chrome/

# Now run your existing code with improved error handling
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import time

# Install chromedriver after ensuring Chrome is installed
chromedriver_autoinstaller.install()

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
# Add user agent to look more like a regular browser
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=options)
# Set longer page load timeout (default is 300 seconds)
driver.set_page_load_timeout(60)

base_url = "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures"
all_links = set()

try:
    print(f"🌐 Attempting to load: {base_url}")
    driver.get(base_url)
    print("✅ Website loaded successfully!")
    
    wait = WebDriverWait(driver, 10)
    
    for page_num in range(1, 11):  # Pages 1 to 10
        print(f"📄 Page {page_num}")

        # Wait for links to render
        wait.until(EC.presence_of_all_elements_located((By.XPATH, '//a[contains(@href, "/knowledge-centre/decisions-and-regulations/regulation-and-procedures/") and string-length(@href) > 70]')))
        time.sleep(1)

        anchors = driver.find_elements(By.XPATH, '//a[contains(@href, "/knowledge-centre/decisions-and-regulations/regulation-and-procedures/") and string-length(@href) > 70]')
        for a in anchors:
            all_links.add(a.get_attribute("href"))

        # Try clicking next page
        if page_num < 10:
            try:
                next_page_link = wait.until(EC.element_to_be_clickable((By.XPATH, f'//a[@aria-label="Page {page_num + 1}"]')))
                driver.execute_script("arguments[0].click();", next_page_link)
                time.sleep(2)
            except Exception as e:
                print(f"❌ Could not click to page {page_num + 1}: {e}")
                break

    print(f"\n✅ Total unique links collected: {len(all_links)}")
    
except Exception as e:
    print(f"❌ Error loading website: {e}")
    print("💡 This might be a network issue or the website is blocking automated requests")
    
finally:
    driver.quit()

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=45559): Read timed out. (read timeout=120)

In [3]:
import os
import requests
from urllib.parse import urljoin, unquote, urlparse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import time
import re

chromedriver_autoinstaller.install()

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

download_folder = r"C:\Users\ali-d\Desktop\KSA\hrsdpdfs"
os.makedirs(download_folder, exist_ok=True)

def clean_filename(text):
    # Remove characters that can't be in filenames
    return re.sub(r'[\\/*?:"<>|]', "", text).strip()

for link in all_links:
    print(f"\n🔍 Visiting: {link}")
    driver.get(link)
    time.sleep(2)

    anchors = driver.find_elements(By.TAG_NAME, "a")
    found = False

    for a in anchors:
        href = a.get_attribute("href")
        if href and href.endswith(".pdf"):
            full_url = urljoin("https://www.hrsd.gov.sa", href)

            # Use aria-label as filename
            aria_label = a.get_attribute("aria-label")
            if aria_label:
                filename = clean_filename(aria_label)
            else:
                # fallback to filename from URL
                filename = os.path.basename(unquote(urlparse(href).path))

            if not filename.endswith(".pdf"):
                filename += ".pdf"

            filepath = os.path.join(download_folder, filename)

            try:
                response = requests.get(full_url, stream=True)
                with open(filepath, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"✅ Downloaded: {filename}")
                found = True
            except Exception as e:
                print(f"❌ Failed to download {full_url}: {e}")

    if not found:
        print("⚠️ No PDFs found.")

driver.quit()


NameError: name 'all_links' is not defined

In [10]:
# Install required Python packages
!pip install python-docx PyMuPDF
# Note: For Windows, we'll use PyMuPDF instead of poppler-utils for better compatibility

import os
from docx import Document

pdf_dir = r"C:\Users\ali-d\Desktop\KSA\hrsdpdfs"
docx_dir = r"C:\Users\ali-d\Desktop\KSA\hrsddocx"
os.makedirs(docx_dir, exist_ok=True)

pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith(".pdf")]

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file)
    
    # Read and write to .docx using basic extraction
    docx_path = os.path.join(docx_dir, pdf_file.replace(".pdf", ".docx"))
    doc = Document()

    try:
        # Use PyMuPDF for text extraction (Windows compatible)
        import fitz
        pdf_document = fitz.open(pdf_path)
        
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            text = page.get_text()
            if text.strip():
                doc.add_paragraph(text.strip())
        
        pdf_document.close()
        doc.save(docx_path)
        print(f"✅ Converted with PyMuPDF: {pdf_file}")
    except Exception as e:
        print(f"❌ Failed to write DOCX for {pdf_file}: {e}")


'pip' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
import re
import os
from docx import Document
from docx.oxml.ns import qn
from docx.enum.text import WD_ALIGN_PARAGRAPH
import fitz # PyMuPDF

pdf_dir = "/content/hrsdpdfs"
docx_dir = "/content/hrsddocx_final"
os.makedirs(docx_dir, exist_ok=True)

# Define common Arabic letters for space re-insertion
ARABIC_LETTERS = r"[\u0600-\u06FF]"
ARABIC_TATWEEL = "\u0640" # Tatweel character

def clean_arabic_text(text):
    # Remove invisible Unicode direction/control chars
    text = re.sub(r"[\u202A-\u202E\u200E\u200F\u2066-\u2069]", "", text)

    # --- Step 1: Normalize common OCR/PDF extraction errors ---
    # Fix common broken Arabic ligatures/forms if they appear due to bad extraction
    # These are very specific and might need tuning based on common errors.
    text = text.replace("ألي", "أي")
    text = text.replace("لألي", "للأ")
    text = text.replace("ءا", "اء")
    text = text.replace("ؤ", "ء")
    text = text.replace("ىء", "ئي") # Common for words like شيء
    text = text.replace("ةال", "ة ال") # If 'ة' merges with 'ال'
    text = text.replace("ااء", "اء") # Like اإليذااء becomes اإليذاء

    # Fix specific problematic words that appear concatenated or malformed
    text = text.replace("نظامحمايةالطفل", "نظام حماية الطفل")
    text = text.replace("المادةاألولى:", "المادة الأولى:")
    text = text.replace("المادةالثانية:", "المادة الثانية:")
    text = text.replace("المادةالثالثة:", "المادة الثالثة:")
    text = text.replace("المادةالرابعة:", "المادة الرابعة:")
    text = text.replace("المادةالخامسة:", "المادة الخامسة:")
    text = text.replace("المادةالسادسة:", "المادة السادسة:")
    text = text.replace("المادةالسابعة:", "المادة السابعة:")
    text = text.replace("المادةالثامنة:", "المادة الثامنة:")
    text = text.replace("المادةالتاسعة:", "المادة التاسعة:")
    text = text.replace("المادةالعاشرة:", "المادة العاشرة:")
    text = text.replace("المادةالحاديةعشرة:", "المادة الحادية عشرة:")
    text = text.replace("المادةالثانيةعشرة:", "المادة الثانية عشرة:")
    text = text.replace("المادةالثالثةعشرة:", "المادة الثالثة عشرة:")
    text = text.replace("المادةالرابعةعشرة:", "المادة الرابعة عشرة:")
    text = text.replace("المادةالخامسةعشرة:", "المادة الخامسة عشرة:")
    text = text.replace("المادةالسادسةعشرة:", "المادة السادسة عشرة:")
    text = text.replace("المادةالسابعةعشرة:", "المادة السابعة عشرة:")
    text = text.replace("المادةالثامنةعشرة:", "المادة الثامنة عشرة:")
    text = text.replace("المادةالتاسعةعشرة:", "المادة التاسعة عشرة:")
    text = text.replace("المادةالعشرون:", "المادة العشرون:")
    text = text.replace("المادةالحاديةوالعشرون:", "المادة الحادية والعشرون:")
    text = text.replace("المادةالثانيةوالعشرون:", "المادة الثانية والعشرون:")
    text = text.replace("المادةالثالثةوالعشرون:", "المادة الثالثة والعشرون:")
    text = text.replace("المادةالرابعةوالعشرون:", "المادة الرابعة والعشرون:")
    text = text.replace("المادةالخامسةوالعشرون:", "المادة الخامسة والعشرون:")

    text = text.replace("الاللائحة", "اللائحة") # Correct typo if it occurs consistently
    text = text.replace("المءسسات", "المؤسسات") # Correct typo for 'مؤسسات'
    text = text.replace("اإليذااء", "الإيذاء") # Correct typo
    text = text.replace("اإلجرااءت", "الإجراءات") # Correct typo
    text = text.replace("االدعااءلعام", "الادعاء العام") # Correct typo
    text = text.replace("االيذااءلواردة", "الإيذاء الواردة") # Correct typo

    # --- Step 2: Intelligent Space Re-insertion ---
    # This regex looks for an Arabic character followed immediately by another Arabic character,
    # where a space should logically be (e.g., between words).
    # It avoids inserting spaces within naturally joined words or around punctuation.
    # It's a heuristic and might require fine-tuning for specific edge cases.
    # This is complex and might lead to over-spacing or under-spacing.
    # A more advanced approach would be to use an Arabic tokenizer if available.

    # Insert spaces before capital English letters if they follow Arabic (e.g., ArabicWordEnglishWord)
    text = re.sub(r"([%s])([A-Z])" % ARABIC_LETTERS, r"\1 \2", text)
    # Insert spaces after English letters if followed by Arabic (e.g., EnglishWordArabicWord)
    text = re.sub(r"([a-zA-Z])([%s])" % ARABIC_LETTERS, r"\1 \2", text)

    # Insert a space if an Arabic letter is followed immediately by another Arabic letter
    # and the first letter is NOT a character that typically joins (e.g., not lam, kaf, etc. if it's the start of a new word)
    # This is a very challenging problem without a dictionary or NLP.
    # A simpler approach: if a non-joining letter is followed by any other Arabic letter, insert space.
    # E.g., 'ة' followed by 'ا', 'و' followed by 'ا', 'ر' followed by 'ا', 'د' followed by 'ا'
    # This needs careful regex construction. For now, let's target specific cases.

    # For `نظامحمايةالطفل`: insert space when `م` is followed by `ح` etc.
    # This will be tricky without knowing Arabic word boundaries.
    # Let's use a simpler heuristic for now: If a sequence of Arabic letters ends, and another begins without a space.
    text = re.sub(r"([%s])(?=[%s])" % (ARABIC_LETTERS, ARABIC_LETTERS), lambda m: m.group(0) + " " if (
        m.group(0)[-1] not in ['ا', 'د', 'ذ', 'ر', 'ز', 'و', 'ة', ARABIC_TATWEEL] and # Non-joining end letters
        re.match(r"[^%s]" % ARABIC_LETTERS, text[m.end():m.end()+1]) is None # Next char is Arabic
    ) else m.group(0), text)
    # The above is prone to errors, often it's better to manually specify common concatenations if limited.
    # Reverting to a more general space insertion if PyMuPDF still concatenates
    # Add a space before any Arabic letter if it's preceded by another Arabic letter that *should* break a word
    # This is a highly complex problem without a robust Arabic tokenizer.
    # Let's try to identify common breaking points.
    text = re.sub(r'(?<![اأإآبتثجحخدذرزسشصضطظعغفقكلمنهويىءئؤ])([اأإآبتثجحخدذرزسشصضطظعغفقكلمنهويىءئؤ])', r' \1', text)
    text = re.sub(r'([اأإآبتثجحخدذرزسشصضطظعغفقكلمنهويىءئؤ])(?![اأإآبتثجحخدذرزسشصضطظعغفقكلمنهويىءئؤ])', r'\1 ', text)
    text = re.sub(r'([اأإآبتثجحخدذرزسشصضطظعغفقكلمنهويىءئؤ])([اأإآبتثجحخدذرزسشصضطظعغفقكلمنهويىءئؤ])', r'\1 \2', text)
    # The last one is very aggressive, might introduce too many spaces.
    # Let's try to be smart about punctuation.
    text = re.sub(r'([%s])([.,:;!?])' % ARABIC_LETTERS, r'\1\2', text) # Remove space before punctuation
    text = re.sub(r'([.,:;!?])([%s])' % ARABIC_LETTERS, r'\1 \2', text) # Add space after punctuation if followed by Arabic

    # --- Step 3: Clean up numbers and formatting markers ---
    # Fix list numbers: ` .12` -> `1.` and ensure arabic digits
    text = re.sub(r"^\s*\.(\d+)", r"\1.", text) # e.g., ".1" -> "1."
    text = re.sub(r"(\d+)\s*\.", r"\1.", text) # Ensure dot is right after number
    text = re.sub(r"^\s*(\d+)", r"\1", text) # Remove leading space before number if no dot
    text = re.sub(r"([.،؛؟])(\d+)", r"\1 \2", text) # Ensure space after punctuation before number
    text = re.sub(r"(\d+)(\s*[.،؛؟])", r"\1\2", text) # Remove space before punctuation after number

    # Convert Western digits to Arabic digits (if not already by PyMuPDF)
    arabic_digits = str.maketrans("0123456789", "٠١٢٣٤٥٦٧٨٩")
    text = text.translate(arabic_digits)

    # Remove single digits or very short lines of non-Arabic chars (potential page numbers/noise)
    if re.fullmatch(r"^\s*\d+\s*$", text): # Lines that are only digits
        return ""
    if re.fullmatch(r"^\s*[^\u0600-\u06FF]{1,5}\s*$", text): # Lines with 1-5 non-Arabic chars (noise)
        return ""

    # Remove multiple spaces
    text = re.sub(r"\s{2,}", " ", text)

    return text.strip()

def set_rtl_paragraph(paragraph):
    paragraph.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    p = paragraph._element
    p.set(qn("w:rtl"), "1")
    # Also set Latin and Arabic font to ensure proper display
    # This might require specific font names installed on the system where DOCX is viewed
    # rPr = p.get_or_add_rPr()
    # rFonts = OxmlElement('w:rFonts')
    # rFonts.set(qn('w:ascii'), 'Arial') # Or a robust Arabic font
    # rFonts.set(qn('w:hAnsi'), 'Arial')
    # rFonts.set(qn('w:cs'), 'Arial')
    # rPr.insert_element_before(rFonts, 'w:rtl')


# Process all PDFs
for pdf_file in os.listdir(pdf_dir):
    if not pdf_file.lower().endswith(".pdf"):
        continue

    pdf_path = os.path.join(pdf_dir, pdf_file)
    docx_path = os.path.join(docx_dir, pdf_file.replace(".pdf", ".docx"))
    doc = Document()

    try:
        # Use PyMuPDF to extract text
        pdf_document = fitz.open(pdf_path)
        full_text_blocks = []
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            # 'text' extracts text blocks, usually better for structure
            # 'words' extracts individual words with their bounding boxes, can be used for more precise re-assembly
            # For now, let's stick with 'text' and then clean line by line or block by block
            text_on_page = page.get_text("text")
            full_text_blocks.append(text_on_page)
        pdf_document.close()

        # Join all text blocks and process
        raw_full_text = "\n".join(full_text_blocks)

        # Remove header/footer noise here BEFORE detailed line processing
        # This regex removes lines that look like page numbers or very short identifiers
        cleaned_raw_text = re.sub(r"^\s*\d+\s*$", "", raw_full_text, flags=re.MULTILINE)
        cleaned_raw_text = re.sub(r"^\s*(www\.hrsd\.gov\.sa|Classi|ﻣﻘٍ|Classi).*$", "", cleaned_raw_text, flags=re.MULTILINE | re.IGNORECASE)

        lines = cleaned_raw_text.splitlines()
        current_paragraph_lines = []

        for line in lines:
            line = line.strip()
            if not line: # Empty line, indicates a paragraph break
                if current_paragraph_lines:
                    # Join lines within the detected paragraph with a space, then clean
                    paragraph_text = " ".join(current_paragraph_lines)
                    cleaned_paragraph = clean_arabic_text(paragraph_text)
                    if cleaned_paragraph:
                        p = doc.add_paragraph(cleaned_paragraph)
                        set_rtl_paragraph(p)
                    current_paragraph_lines = [] # Reset for next paragraph
                continue

            # Check for bullet points or numbered lists at the start of the line.
            # If a line clearly starts a new item, treat it as a new paragraph.
            is_new_list_item = False
            # Check for common list item markers like numbers, bullet characters, or specific Arabic list patterns
            if re.match(r"^\s*([٠-٩0-9]+\.|\*|•)\s*", line): # Arabic/Western numbers followed by dot, or common bullets
                 is_new_list_item = True
            elif re.match(r"^\s*([أ-ي] -|-)\s*", line): # Arabic letters followed by dash, or just a dash
                 is_new_list_item = True


            # If it's a new list item, or looks like a new heading, create a new paragraph for the current content
            if is_new_list_item and current_paragraph_lines:
                paragraph_text = " ".join(current_paragraph_lines)
                cleaned_paragraph = clean_arabic_text(paragraph_text)
                if cleaned_paragraph:
                    p = doc.add_paragraph(cleaned_paragraph)
                    set_rtl_paragraph(p)
                current_paragraph_lines = []

            # Append the current line to the paragraph, then let clean_arabic_text handle internal spacing
            current_paragraph_lines.append(line)

        # Add any remaining text as a final paragraph
        if current_paragraph_lines:
            paragraph_text = " ".join(current_paragraph_lines)
            cleaned_paragraph = clean_arabic_text(paragraph_text)
            if cleaned_paragraph:
                p = doc.add_paragraph(cleaned_paragraph)
                set_rtl_paragraph(p)

        doc.save(docx_path)
        print(f"✅ Saved: {os.path.basename(docx_path)}")

    except Exception as e:
        print(f"❌ Error processing {pdf_file}: {e}")

✅ Saved: 11102022.docx
✅ Saved: L192891.docx
✅ Saved: أطر العمل التنظيمية للائحة التنفيذية للموارد البشرية.docx
✅ Saved: إدارة مكافحة جرائم الاتجار بالأشخاص.docx
✅ Saved: أطر العمل التنظيمية للائحة التنفيذية للموارد البشرية.docx
✅ Saved: إدارة مكافحة جرائم الاتجار بالأشخاص.docx
✅ Saved: إطار التدريب والابتعاث والإيفاد للعام المالي 2026م.docx
✅ Saved: إطار العمل التنظيمي للائحة الوظائف التعليمية.docx
✅ Saved: اشتراط التسجيل المهني للمحاسبين الوافدين.docx
✅ Saved: إطار التدريب والابتعاث والإيفاد للعام المالي 2026م.docx
✅ Saved: إطار العمل التنظيمي للائحة الوظائف التعليمية.docx
✅ Saved: اشتراط التسجيل المهني للمحاسبين الوافدين.docx
✅ Saved: اشتراطات بيئة العمل لقطاع الحراسات الأمنية.docx
✅ Saved: اشتراطات بيئة العمل لقطاع الحراسات الأمنية.docx
✅ Saved: الأسئلة الشائعة لانتقال مستفيدي الضمان السابق للضمان الاجتماعي المطور.docx
✅ Saved: الأسئلة الشائعة لبرنامج نطاقات المطور.docx
✅ Saved: الأسئلة الشائعة لانتقال مستفيدي الضمان السابق للضمان الاجتماعي المطور.docx
✅ Saved: الأسئلة الشائعة 

In [12]:
# First, run these commands in a Colab cell to install the necessary libraries:
# !pip install python-docx
# !pip install PyMuPDF

import re
import os
from docx import Document
from docx.oxml.ns import qn
from docx.enum.text import WD_ALIGN_PARAGRAPH
import fitz # PyMuPDF

# --- Configuration ---
# IMPORTANT:
# 1. PDF files should be in the 'hrsdpdfs' folder in your workspace directory.
#    The script will automatically create this folder if it doesn't exist.
# 2. The script will create a folder named 'hrsddocx_final' to save the final processed DOCX files.
# 3. All output folders will be created in: C:\Users\ali-d\Desktop\KSA\
pdf_dir = r"C:\Users\ali-d\Desktop\KSA\hrsdpdfs"
docx_dir = r"C:\Users\ali-d\Desktop\KSA\hrsddocx_final"
os.makedirs(pdf_dir, exist_ok=True)
os.makedirs(docx_dir, exist_ok=True)

# Define common Arabic letters and Tatweel for regex patterns
ARABIC_LETTERS = r"[\u0600-\u06FF]"
ARABIC_TATWEEL = "\u0640" # Tatweel character (elongation mark)

def clean_arabic_text(text):
    # Remove invisible Unicode direction/control characters that can interfere with display
    text = re.sub(r"[\u202A-\u202E\u200E\u200F\u2066-\u2069]", "", text)

    # --- Remove specific parenthesis characters (and any spaces around them) ---
    text = re.sub(r'\s*[\(\)]\s*', '', text) # This will remove both ( and ) and any surrounding spaces

    # --- Step 1: Normalize common text extraction errors and specific word issues ---
    # Fix common broken Arabic ligatures/forms if they appear due to bad extraction
    text = text.replace("ألي", "أي")
    text = text.replace("لألي", "للأ")
    text = text.replace("ءا", "اء")
    text = text.replace("ؤ", "ء")
    text = text.replace("ىء", "ئي") # Common for words like شيء
    text = text.replace("ةال", "ة ال") # If 'ة' (taa marbutah) merges with 'ال' (alif lam)
    text = text.replace("ااء", "اء") # Corrects patterns like اإليذااء to اإليذاء

    # Fix specific problematic words that appeared concatenated or malformed in your samples
    # You can add more of these specific replacements as you identify recurring issues
    text = text.replace("نظامحمايةالطفل", "نظام حماية الطفل")
    text = text.replace("المادةاألولى:", "المادة الأولى:")
    text = text.replace("المادةالثانية:", "المادة الثانية:")
    text = text.replace("المادةالثالثة:", "المادة الثالثة:")
    text = text.replace("المادةالرابعة:", "المادة الرابعة:")
    text = text.replace("المادةالخامسة:", "المادة الخامسة:")
    text = text.replace("المادةالسادسة:", "المادة السادسة:")
    text = text.replace("المادةالسابعة:", "المادة السابعة:")
    text = text.replace("المادةالثامنة:", "المادة الثامنة:")
    text = text.replace("المادةالتاسعة:", "المادة التاسعة:")
    text = text.replace("المادةالعاشرة:", "المادة العاشرة:")
    text = text.replace("المادةالحاديةعشرة:", "المادة الحادية عشرة:")
    text = text.replace("المادةالثانيةعشرة:", "المادة الثانية عشرة:")
    text = text.replace("المادةالثالثةعشرة:", "المادة الثالثة عشرة:")
    text = text.replace("المادةالرابعةعشرة:", "المادة الرابعة عشرة:")
    text = text.replace("المادةالخامسةعشرة:", "المادة الخامسة عشرة:")
    text = text.replace("المادةالسادسةعشرة:", "المادة السادسة عشرة:")
    text = text.replace("المادةالسابعةعشرة:", "المادة السابعة عشرة:")
    text = text.replace("المادةالثامنةعشرة:", "المادة الثامنة عشرة:")
    text = text.replace("المادةالتاسعةعشرة:", "المادة التاسعة عشرة:")
    text = text.replace("المادةالعشرون:", "المادة العشرون:")
    text = text.replace("المادةالحاديةوالعشرون:", "المادة الحادية والعشرون:")
    text = text.replace("المادةالثانيةوالعشرون:", "المادة الثانية والعشرون:")
    text = text.replace("المادةالثالثةوالعشرون:", "المادة الثالثة والعشرون:")
    text = text.replace("المادةالرابعةوالعشرون:", "المادة الرابعة والعشرون:")
    text = text.replace("المادةالخامسةوالعشرون:", "المادة الخامسة والعشرون:")
    text = text.replace("الاللائحة", "اللائحة") # Correct typo if it occurs consistently
    text = text.replace("المءسسات", "المؤسسات") # Correct typo for 'مؤسسات'
    text = text.replace("اإليذااء", "الإيذاء") # Correct typo
    text = text.replace("اإلجرااءت", "الإجراءات") # Correct typo
    text = text.replace("االدعااءلعام", "الادعاء العام") # Correct typo
    text = text.replace("االيذااءلواردة", "الإيذاء الواردة") # Correct typo
    text = text.replace("حاجاتهااألساسية", "حاجاتها الأساسية") # Specific concatenation
    text = text.replace("بعدتسعينيومً", "بعد تسعين يومً") # Fix for "بعد تسعين يومً" concatenation

    # --- NEW: Replacements for the provided list of "troubling words" ---
    text = text.replace("التـأكيـد", "التأكيد")
    text = text.replace("الشــريعـة الإســلاميـة", "الشريعة الإسلامية")
    text = text.replace("والاتفـاقيـات", "والاتفاقيات")
    text = text.replace("أشـكال", "أشكال")
    text = text.replace("بهالمنزل", "به المنزل")
    text = text.replace("المدرسـة", "المدرسة")
    text = text.replace("والأهليـة", "والأهلية")
    text = text.replace("حكمهـاســواء", "حكمها سواء")
    text = text.replace("ذلـك", "ذلك")
    text = text.replace("مسءولية", "مسؤولية")
    text = text.replace("والإهمال؛بتوفير", "والإهمال؛ بتوفير")
    text = text.replace("تءدي", "تؤدي")
    text = text.replace("مءسسات", "مؤسسات")
    text = text.replace("تش ـغيل", "تشغيل")
    text = text.replace("عش ـرة،كما", "عشرة، كما")
    text = text.replace("المءثرات", "المؤثرات")
    text = text.replace("يسـتخدم", "يستخدم")
    text = text.replace("شـرائها", "شرائها")
    text = text.replace("رقمم", "رقم")
    text = text.replace("انتاج ونشـر", "إنتاج ونشر")
    text = text.replace("مصـنف", "مصنف")
    text = text.replace("مسـموع", "مسموع")
    text = text.replace("او الفكري", "أو الفكري")
    text = text.replace("والمسءولية", "والمسؤولية")
    text = text.replace("مسـءولين", "مسؤولين")
    text = text.replace("وضـمان", "وضمان")
    text = text.replace("لضـمان", "لضمان")
    text = text.replace("مسـءولياتهما", "مسؤولياتهما")
    text = text.replace("ســرعة", "سرعة")
    text = text.replace("والإصــلاح المناســبة", "والإصلاح المناسبة")
    text = text.replace("ســلامته", "سلامته")
    text = text.replace("النفســية", "النفسية")
    text = text.replace("الوقـايـة والإرشـــاد الصــحي والتوعيـة", "الوقاية والإرشاد الصحي والتوعية")
    text = text.replace("بصــحتـه وتغـذيتـه ومزايـا", "بصحته وتغذيته ومزايا")
    text = text.replace("يعيشــون", "يعيشون")
    text = text.replace("صــعبة،كالأطفال", "صعبة، كالأطفال")
    text = text.replace("والمشــردين،وضــحايا", "والمشردين، وضحايا")
    text = text.replace("الملكي رقمم/", "الملكي رقم/")
    text = text.replace("صـحية", "صحية")
    text = text.replace("ونفسـية", "ونفسية")
    text = text.replace("الصــحة،لإخضــاع", "الصحة، لإخضاع")
    text = text.replace("او", "أو")
    text = text.replace("وضـع", "وضع")
    text = text.replace("شـاملة", "شاملة")
    text = text.replace("المصـنعة", "المصنعة")
    text = text.replace("المسـتوردة،بحيث", "المستوردة، بحيث")
    text = text.replace("للمواصـفات", "للمواصفات")
    text = text.replace("والإجراءت", "والإجراءات")
    text = text.replace("إجراءت", "إجراءات")
    text = text.replace("مراعـاة مـا", "مراعاة ما")
    text = text.replace("والعشــرينمن", "والعشرين من")
    text = text.replace("هيئـة", "هيئة")
    text = text.replace("والادعـاء العـام", "والادعاء العام")
    text = text.replace("اخلال", "إخلال")
    text = text.replace("المـادة", "المادة")
    text = text.replace("والعشــرين مكررمن", "والعشرين مكرر من")
    text = text.replace("مراعـاة الأحكـام", "مراعاة الأحكام")
    text = text.replace("الإجراءت", "الإجراءات")
    text = text.replace("البشــريـة", "البشرية")
    text = text.replace("الاجتمـاعيـة متـابعـة", "الاجتماعية متابعة")
    text = text.replace("اليهـا", "إليها")
    text = text.replace("تريبهاالثالثة", "ترتيبها الثالثة")
    text = text.replace("مكررلتكون", "مكرر لتكون")
    text = text.replace("علىســنتينوبغرامـة", "على سنتين وبغرامة")
    text = text.replace("علىمـائـةألف", "على مائة ألف")
    text = text.replace("بـإحـدى هـاتين العقوبتين،كـل", "بإحدى هاتين العقوبتين، كل")
    text = text.replace("ارتكـب", "ارتكب")
    text = text.replace("شــكـل", "شكل")
    text = text.replace("أفعـال الايـذاء", "أفعال الإيذاء")
    text = text.replace("الأولىمن", "الأولى من")
    text = text.replace("تقـل", "تقل")
    text = text.replace("علىخ خمسســ سنوات،وغرامة", "على خمس سنوات، وغرامة")
    text = text.replace("عنمائةألف", "عن مائة ألف")
    text = text.replace("علىخمسمائةألف", "على خمسمائة ألف")
    text = text.replace("الايذاء", "الإيذاء")
    text = text.replace("سـاعده", "ساعده")
    text = text.replace("صـورة", "صورة")
    text = text.replace("المسـاعدة", "المساعدة")
    text = text.replace("المشـار", "المشار")
    text = text.replace("خلالتسـعينيومً", "خلال تسعين يومً")

    # --- Step 2: Handle spacing around punctuation and between English/Arabic text ---
    # Consolidate punctuation: remove spaces immediately before/after punctuation
    text = re.sub(r'\s*([.,:;!?،؛؟])\s*', r'\1', text)
    # Ensure a single space *after* punctuation if it's followed by an Arabic letter
    text = re.sub(r'([.,:;!?،؛؟])([%s])' % ARABIC_LETTERS, r'\1 \2', text)

    # Insert spaces before capital English letters if they follow Arabic (e.g., ArabicWordEnglishWord)
    text = re.sub(r"([%s])([A-Z])" % ARABIC_LETTERS, r"\1 \2", text)
    # Insert spaces after English letters if followed by Arabic (e.g., EnglishWordArabicWord)
    text = re.sub(r"([a-zA-Z])([%s])" % ARABIC_LETTERS, r"\1 \2", text)

    # IMPORTANT: The overly aggressive general space insertion rules that caused "every letter has space"
    # have been intentionally removed from this version.
    # We are relying on PyMuPDF's base extraction and specific replacements for concatenation.


    # --- Step 3: Clean up numbers and other formatting noise ---
    # Fix list numbers: ` .12` -> `1.` and ensure correct Arabic digits
    text = re.sub(r"^\s*\.(\d+)", r"\1.", text) # e.g., ".1" -> "1." at start of line
    text = re.sub(r"(\d+)\s*\.", r"\1.", text) # Ensure dot is right after number
    text = re.sub(r"^\s*(\d+)", r"\1", text) # Remove leading space before number if no dot/punctuation
    text = re.sub(r"([.،؛؟])(\d+)", r"\1 \2", text) # Ensure space after punctuation before number
    text = re.sub(r"(\d+)(\s*[.،؛؟])", r"\1\2", text) # Remove space before punctuation after number

    # Convert Western digits to Arabic digits (if not already handled by PyMuPDF extraction)
    arabic_digits = str.maketrans("0123456789", "٠١٢٣٤٥٦٧٨٩")
    text = text.translate(arabic_digits)

    # Remove lines that are likely page numbers or very short non-Arabic noise
    # Apply these only if the *entire* line matches the pattern
    if re.fullmatch(r"^\s*\d+\s*$", text): # Lines that are only digits (e.g., page numbers)
        return ""
    if re.fullmatch(r"^\s*[^\u0600-\u06FF]{1,5}\s*$", text): # Lines with 1-5 non-Arabic chars (potential noise)
        return ""

    # Collapse multiple spaces into a single space
    text = re.sub(r"\s{2,}", " ", text)

    return text.strip()

def set_rtl_paragraph(paragraph):
    """Sets a paragraph to Right-To-Left (RTL) alignment in DOCX."""
    paragraph.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    p = paragraph._element
    p.set(qn("w:rtl"), "1")
    # You might also want to set font properties here if specific Arabic fonts
    # are desired and installed on the user's system viewing the DOCX.
    # For example:
    # rPr = p.get_or_add_rPr()
    # rFonts = OxmlElement('w:rFonts')
    # rFonts.set(qn('w:ascii'), 'Arial') # Or 'Times New Roman'
    # rFonts.set(qn('w:hAnsi'), 'Arial')
    # rFonts.set(qn('w:cs'), 'Arial') # This is for complex scripts like Arabic
    # rPr.insert_element_before(rFonts, 'w:rtl')


# --- Main Processing Loop ---
print(f"Starting PDF processing from {pdf_dir}...")
for pdf_file in os.listdir(pdf_dir):
    if not pdf_file.lower().endswith(".pdf"):
        continue # Skip non-PDF files

    pdf_path = os.path.join(pdf_dir, pdf_file)
    docx_path = os.path.join(docx_dir, pdf_file.replace(".pdf", ".docx"))
    doc = Document() # Create a new DOCX document for each PDF

    try:
        # Use PyMuPDF to extract text from the PDF
        pdf_document = fitz.open(pdf_path)
        full_text_blocks = []
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            # 'text' extraction method usually preserves some structural information
            text_on_page = page.get_text("text")
            full_text_blocks.append(text_on_page)
        pdf_document.close()

        # Join all extracted page texts into one raw string
        raw_full_text = "\n".join(full_text_blocks)

        # Remove header/footer noise and other general unwanted lines before detailed paragraph processing
        cleaned_raw_text = re.sub(r"^\s*\d+\s*$", "", raw_full_text, flags=re.MULTILINE) # Removes lines with only numbers
        cleaned_raw_text = re.sub(r"^\s*(www\.hrsd\.gov\.sa|Classi|ﻣﻘٍ|Classi).*$", "", cleaned_raw_text, flags=re.MULTILINE | re.IGNORECASE) # Removes specific footer/header texts

        lines = cleaned_raw_text.splitlines()
        current_paragraph_lines = []

        for line in lines:
            line = line.strip() # Remove leading/trailing whitespace from each line
            if not line: # An empty line often indicates a paragraph break in PDFs
                if current_paragraph_lines: # If there's content accumulated for a paragraph
                    # Join lines within the detected paragraph with a space, then clean
                    paragraph_text = " ".join(current_paragraph_lines)
                    cleaned_paragraph = clean_arabic_text(paragraph_text)
                    if cleaned_paragraph: # Only add if content remains after cleaning
                        p = doc.add_paragraph(cleaned_paragraph)
                        set_rtl_paragraph(p)
                    current_paragraph_lines = [] # Reset for the next paragraph
                continue # Move to the next line

            # Heuristic to detect new list items or section headings
            # This helps to avoid merging list items or distinct headings into previous paragraphs.
            is_new_list_item = False
            # Check for common list item markers (Arabic/Western numbers, bullets)
            if re.match(r"^\s*([٠-٩0-9]+\.|\*|•)\s*", line):
                 is_new_list_item = True
            # Check for Arabic letters followed by a dash, or just a dash (common in Arabic lists/sub-sections)
            elif re.match(r"^\s*([أ-ي] -|-)\s*", line):
                 is_new_list_item = True

            # If a new list item or heading is detected and we have existing lines for a paragraph,
            # finalize the current paragraph before starting a new one.
            if is_new_list_item and current_paragraph_lines:
                paragraph_text = " ".join(current_paragraph_lines)
                cleaned_paragraph = clean_arabic_text(paragraph_text)
                if cleaned_paragraph:
                    p = doc.add_paragraph(cleaned_paragraph)
                    set_rtl_paragraph(p)
                current_paragraph_lines = []

            # Add the current line to the current paragraph's lines
            current_paragraph_lines.append(line)

        # After the loop, add any remaining accumulated text as the final paragraph
        if current_paragraph_lines:
            paragraph_text = " ".join(current_paragraph_lines)
            cleaned_paragraph = clean_arabic_text(paragraph_text)
            if cleaned_paragraph:
                p = doc.add_paragraph(cleaned_paragraph)
                set_rtl_paragraph(p)

        doc.save(docx_path) # Save the generated DOCX file
        print(f"✅ Successfully converted: {os.path.basename(pdf_file)} to {os.path.basename(docx_path)}")

    except Exception as e:
        print(f"❌ Error processing {pdf_file}: {e}")

print("Processing complete.")

Starting PDF processing from C:\Users\ali-d\Desktop\KSA\hrsdpdfs...
Processing complete.


In [14]:
print("Collected links:")
for i, link in enumerate(all_links, 1):
    print(f"{i}. {link}")
print(f"\nTotal: {len(all_links)} links")

Collected links:
1. https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/violations
2. https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/evidence
3. https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/system
4. https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/regulation
5. https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/procedure

Total: 5 links


In [15]:
# Let's debug the website structure to see what we're actually getting
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox") 
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

base_url = "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures"
print(f"🌐 Visiting: {base_url}")
driver.get(base_url)
time.sleep(3)

print("\n🔍 Looking for all links on the page...")
all_links_on_page = driver.find_elements(By.TAG_NAME, "a")
print(f"Total links found: {len(all_links_on_page)}")

print("\n📋 Sample of links (first 20):")
for i, link in enumerate(all_links_on_page[:20]):
    href = link.get_attribute("href")
    text = link.text.strip()[:50] if link.text else "No text"
    if href:
        print(f"{i+1}. {text} -> {href}")

driver.quit()

🌐 Visiting: https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures

🔍 Looking for all links on the page...
Total links found: 276

📋 Sample of links (first 20):
1. No text -> https://app-na.readspeaker.com/cgi-bin/rsent?customerid=8034&lang=ar_ar&readid=main&url=https://www.hrsd.gov.sa/ar/knowledge-centre/decisions-and-regulations/regulation-and-procedures
2. Skip to main content -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures#main-content
3. No text -> https://www.hrsd.gov.sa/
4. No text -> https://www.hrsd.gov.sa/ministry
5. No text -> https://www.hrsd.gov.sa/ministry/about-ministry
6. No text -> https://www.hrsd.gov.sa/ministry/about-ministry/about-us
7. No text -> https://www.hrsd.gov.sa/ministry/about-ministry/organizational-chart
8. No text -> https://www.hrsd.gov.sa/ministry/about-ministry/policies-strategies
9. No text -> https://www.hrsd.gov.sa/ministry/about-ministry/budget-expenses
10. No t

In [16]:
# Look for document links more specifically
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

base_url = "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures"
driver.get(base_url)
time.sleep(3)

print("🔍 Looking for document/regulation links...")

# Look for links that might lead to documents
potential_doc_links = []
all_links = driver.find_elements(By.TAG_NAME, "a")

for link in all_links:
    href = link.get_attribute("href")
    text = link.text.strip() if link.text else ""
    
    if href and ("regulation" in href or "procedure" in href or "decision" in href):
        # Skip navigation/menu links and look for content links
        if len(href) > 50 and "knowledge-centre" in href:
            potential_doc_links.append((text[:80], href))

print(f"Found {len(potential_doc_links)} potential document links:")
for i, (text, href) in enumerate(potential_doc_links[:10]):
    print(f"{i+1}. {text} -> {href}")

if len(potential_doc_links) > 10:
    print(f"... and {len(potential_doc_links) - 10} more")

driver.quit()

🔍 Looking for document/regulation links...
Found 18 potential document links:
1.  -> https://app-na.readspeaker.com/cgi-bin/rsent?customerid=8034&lang=ar_ar&readid=main&url=https://www.hrsd.gov.sa/ar/knowledge-centre/decisions-and-regulations/regulation-and-procedures
2. Skip to main content -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures#main-content
3.  -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations
4.  -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures
5.  -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/ministerial-decisions
6.  -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations
7.  -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures
8.  -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/ministerial-decisions
9.  -> https://www.hrsd.gov.sa/knowledge-centre/decisions-

In [17]:
# Check one of the collected category links to see its structure
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

# Check the first collected link
test_url = "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/violations"
print(f"🔍 Investigating: {test_url}")
driver.get(test_url)
time.sleep(3)

print("\n📄 Looking for PDF links on this page...")
pdf_links = []
all_links = driver.find_elements(By.TAG_NAME, "a")

for link in all_links:
    href = link.get_attribute("href")
    text = link.text.strip() if link.text else ""
    
    if href:
        if href.endswith(".pdf"):
            pdf_links.append((text[:80], href))
        elif "pdf" in href.lower():
            pdf_links.append((f"[MAYBE PDF] {text[:80]}", href))

print(f"Found {len(pdf_links)} PDF-related links:")
for text, href in pdf_links[:10]:
    print(f"- {text} -> {href}")

# Also check for document links that might lead to PDFs
print(f"\n🔍 Looking for document detail pages...")
doc_detail_links = []
for link in all_links:
    href = link.get_attribute("href")
    text = link.text.strip() if link.text else ""
    
    if href and len(text) > 10:  # Links with substantial text
        if ("violation" in href or "regulation" in href) and href != test_url:
            doc_detail_links.append((text[:80], href))

print(f"Found {len(doc_detail_links)} potential document detail pages:")
for text, href in doc_detail_links[:5]:
    print(f"- {text} -> {href}")

driver.quit()

🔍 Investigating: https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/violations

📄 Looking for PDF links on this page...
Found 0 PDF-related links:

🔍 Looking for document detail pages...
Found 4 potential document detail pages:
- Skip to main content -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/violations#main-content
- القرارات والأنظمة -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations
- اللوائح والأنظمة والأدلة -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures
- جدول المخالفات والعقوبات -> https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/%D9%82%D8%B1%D8%A7%D8%B1-%D8%AC%D8%AF%D9%88%D9%84-%D8%A7%D9%84%D9%85%D8%AE%D8%A7%D9%84%D9%81%D8%A7%D8%AA-%D9%88%D8%A7%D9%84%D8%B9%D9%82%D9%88%D8%A8%D8%A7%D8%AA


In [18]:
# Check the specific document page for PDFs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

# Check the document detail page
doc_url = "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/%D9%82%D8%B1%D8%A7%D8%B1-%D8%AC%D8%AF%D9%88%D9%84-%D8%A7%D9%84%D9%85%D8%AE%D8%A7%D9%84%D9%81%D8%A7%D8%AA-%D9%88%D8%A7%D9%84%D8%B9%D9%82%D9%88%D8%A8%D8%A7%D8%AA"
print(f"🔍 Checking document page: {doc_url}")
driver.get(doc_url)
time.sleep(3)

print("\n📄 Looking for PDF links...")
pdf_links = []
all_links = driver.find_elements(By.TAG_NAME, "a")

for link in all_links:
    href = link.get_attribute("href")
    text = link.text.strip() if link.text else ""
    aria_label = link.get_attribute("aria-label")
    
    if href:
        if href.endswith(".pdf"):
            pdf_links.append({
                'text': text[:100],
                'href': href,
                'aria_label': aria_label
            })
            
print(f"Found {len(pdf_links)} PDF links:")
for i, pdf in enumerate(pdf_links):
    print(f"{i+1}. Text: '{pdf['text']}'")
    print(f"   Aria-label: '{pdf['aria_label']}'")
    print(f"   URL: {pdf['href']}")
    print()

# Also look for download buttons or links
print("🔍 Looking for download-related elements...")
download_elements = driver.find_elements(By.XPATH, "//*[contains(text(), 'تحميل') or contains(text(), 'تنزيل') or contains(text(), 'Download') or contains(@class, 'download')]")
for element in download_elements[:5]:
    print(f"- {element.tag_name}: {element.text} -> {element.get_attribute('href') or 'No href'}")

driver.quit()

🔍 Checking document page: https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/%D9%82%D8%B1%D8%A7%D8%B1-%D8%AC%D8%AF%D9%88%D9%84-%D8%A7%D9%84%D9%85%D8%AE%D8%A7%D9%84%D9%81%D8%A7%D8%AA-%D9%88%D8%A7%D9%84%D8%B9%D9%82%D9%88%D8%A8%D8%A7%D8%AA

📄 Looking for PDF links...
Found 3 PDF links:
1. Text: 'قرار جدول المخالفات والعقوبات'
   Aria-label: 'قرار جدول المخالفات والعقوبات'
   URL: https://www.hrsd.gov.sa/sites/default/files/2023-12/%D9%82%D8%B1%D8%A7%D8%B1%20%D8%AC%D8%AF%D9%88%D9%84%20%D8%A7%D9%84%D9%85%D8%AE%D8%A7%D9%84%D9%81%D8%A7%D8%AA%20%D9%88%D8%A7%D9%84%D8%B9%D9%82%D9%88%D8%A8%D8%A7%D8%AA.pdf

2. Text: 'قرار تحديث لجدول المخالفات والعقوبات'
   Aria-label: 'قرار تحديث لجدول المخالفات والعقوبات'
   URL: https://www.hrsd.gov.sa/sites/default/files/2025-03/%D8%AA%D8%AD%D8%AF%D9%8A%D8%AB%20%D9%84%D8%AC%D8%AF%D9%88%D9%84%20%D8%A7%D9%84%D9%85%D8%AE%D8%A7%D9%84%D9%81%D8%A7%D8%AA%20%D9%88%D8%A7%D9%84%D8%B9%D9%82%D9%88%D8%A8%D8%A7%D8%AA.pdf

3. Text: '

In [19]:
# FIXED VERSION - Properly collect document links and PDFs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import requests
from urllib.parse import urljoin, unquote, urlparse
import re

# Setup Chrome driver
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

# Create download folder
download_folder = r"C:\Users\ali-d\Desktop\KSA\hrsdpdfs"
os.makedirs(download_folder, exist_ok=True)

def clean_filename(text):
    return re.sub(r'[\\/*?:"<>|]', "", text).strip()

print("🚀 Starting comprehensive HRSD document collection...")
print("=" * 60)

# Step 1: Get all category pages
base_url = "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures"
print(f"🌐 Visiting base page: {base_url}")
driver.get(base_url)
time.sleep(3)

# Look for category links (the 5 we found earlier)
category_links = set()
anchors = driver.find_elements(By.XPATH, '//a[contains(@href, "/knowledge-centre/decisions-and-regulations/regulation-and-procedures/") and string-length(@href) > 70]')
for a in anchors:
    href = a.get_attribute("href")
    if href and "regulation-and-procedures/" in href and href.count("/") > 5:
        category_links.add(href)

print(f"📂 Found {len(category_links)} category pages:")
for i, link in enumerate(category_links, 1):
    print(f"  {i}. {link.split('/')[-1]}")

# Step 2: Visit each category page to get document detail pages
document_detail_links = set()
for category_url in category_links:
    print(f"\n📖 Exploring category: {category_url}")
    driver.get(category_url)
    time.sleep(3)
    
    # Look for document detail links on this category page
    doc_links = driver.find_elements(By.XPATH, '//a[contains(@href, "/knowledge-centre/decisions-and-regulations/regulation-and-procedures/")]')
    
    for link in doc_links:
        href = link.get_attribute("href")
        text = link.text.strip()
        
        if href and len(text) > 10:  # Links with meaningful text
            # Make sure it's a document detail page (not navigation)
            if href != category_url and href.count("/") > 6:
                document_detail_links.add(href)
                print(f"  📄 Found document: {text[:50]}...")

print(f"\n✅ Found {len(document_detail_links)} document detail pages")

# Step 3: Visit each document detail page to get PDFs
pdf_count = 0
for doc_url in document_detail_links:
    print(f"\n🔍 Checking document: {doc_url.split('/')[-1][:50]}...")
    driver.get(doc_url)
    time.sleep(2)
    
    # Look for PDF links on this page
    pdf_links = driver.find_elements(By.XPATH, '//a[contains(@href, ".pdf")]')
    
    for pdf_link in pdf_links:
        href = pdf_link.get_attribute("href")
        aria_label = pdf_link.get_attribute("aria-label") or pdf_link.text.strip()
        
        if href and href.endswith(".pdf"):
            full_url = urljoin("https://www.hrsd.gov.sa", href)
            
            # Create filename
            if aria_label:
                filename = clean_filename(aria_label)
            else:
                filename = os.path.basename(unquote(urlparse(href).path))
            
            if not filename.endswith(".pdf"):
                filename += ".pdf"
            
            filepath = os.path.join(download_folder, filename)
            
            # Download the PDF
            try:
                response = requests.get(full_url, stream=True)
                with open(filepath, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                pdf_count += 1
                print(f"  ✅ Downloaded: {filename}")
                
            except Exception as e:
                print(f"  ❌ Failed to download {full_url}: {e}")

driver.quit()
print(f"\n🎉 Collection complete! Downloaded {pdf_count} PDF files to: {download_folder}")

🚀 Starting comprehensive HRSD document collection...
🌐 Visiting base page: https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures
📂 Found 5 category pages:
  1. violations
  2. evidence
  3. system
  4. regulation
  5. procedure

📖 Exploring category: https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/violations

📖 Exploring category: https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/evidence

📖 Exploring category: https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/system

📖 Exploring category: https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/regulation

📖 Exploring category: https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/procedure

✅ Found 0 document detail pages

🎉 Collection complete! Downloaded 0 PDF files to: C:\Users\ali-d\Deskto

In [1]:
# IMPROVED VERSION - More careful link detection
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import os
import requests
from urllib.parse import urljoin, unquote, urlparse
import re

# Setup
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

download_folder = r"C:\Users\ali-d\Desktop\KSA\hrsdpdfs"
os.makedirs(download_folder, exist_ok=True)

def clean_filename(text):
    return re.sub(r'[\\/*?:"<>|]', "", text).strip()

print("🔧 Improved HRSD document collection strategy...")
print("=" * 60)

# First, let's manually check each category we know has documents
category_pages = [
    "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/violations",
    "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/evidence", 
    "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/system",
    "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/regulation",
    "https://www.hrsd.gov.sa/knowledge-centre/decisions-and-regulations/regulation-and-procedures/procedure"
]

all_doc_detail_urls = []
pdf_count = 0

for category_url in category_pages:
    category_name = category_url.split('/')[-1]
    print(f"\n📂 Exploring category: {category_name}")
    driver.get(category_url)
    time.sleep(3)
    
    # Get all links on this category page
    links = driver.find_elements(By.TAG_NAME, "a")
    doc_links_found = 0
    
    for link in links:
        href = link.get_attribute("href")
        text = link.text.strip() if link.text else ""
        
        if href and text and len(text) > 5:  # Links with meaningful text
            # Check if this looks like a document detail page
            if (category_name in href and 
                href != category_url and 
                "knowledge-centre" in href and
                len(href) > 80):  # Longer URLs are likely document pages
                
                all_doc_detail_urls.append(href)
                doc_links_found += 1
                print(f"  📄 {text[:60]}...")
    
    print(f"  Found {doc_links_found} document links in {category_name}")

print(f"\n✅ Total document detail pages found: {len(all_doc_detail_urls)}")

# Now visit each document detail page to get PDFs
for doc_url in all_doc_detail_urls:
    print(f"\n🔍 Visiting: {doc_url}")
    driver.get(doc_url)
    time.sleep(3)
    
    # Find PDF links
    pdf_links = driver.find_elements(By.XPATH, '//a[contains(@href, ".pdf")]')
    
    for pdf_link in pdf_links:
        href = pdf_link.get_attribute("href")
        aria_label = pdf_link.get_attribute("aria-label")
        text = pdf_link.text.strip()
        
        if href and href.endswith(".pdf"):
            # Use aria-label first, then text, then filename from URL
            if aria_label:
                filename = clean_filename(aria_label)
            elif text:
                filename = clean_filename(text)
            else:
                filename = os.path.basename(unquote(urlparse(href).path))
            
            if not filename.endswith(".pdf"):
                filename += ".pdf"
            
            filepath = os.path.join(download_folder, filename)
            
            try:
                print(f"  📥 Downloading: {filename}")
                response = requests.get(href, stream=True)
                
                with open(filepath, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                pdf_count += 1
                print(f"  ✅ Success: {filename}")
                
            except Exception as e:
                print(f"  ❌ Failed: {e}")

driver.quit()
print(f"\n🎉 Final result: {pdf_count} PDF files downloaded to {download_folder}")

🔧 Improved HRSD document collection strategy...

📂 Exploring category: violations


KeyboardInterrupt: 